In [2]:
!pip install nlp
!pip install transformers
!pip install transformers-interpret
!pip install textattack
!pip install tensorflow_hub

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import torch
import pandas as pd
import scipy as sp
import numpy as np
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
train = pd.read_csv("./data/medical_tc_train_cleaned.csv")
test =  pd.read_csv("./data/medical_tc_test_cleaned.csv")
labels = pd.read_csv("./data/medical_tc_labels.csv")

In [6]:
print(test.head())

      condition_label                                   medical_abstract
0  non-cardiovascular  obstructive sleep apnea following topical orop...
1  non-cardiovascular  neutrophil function and pyogenic infections bo...
2  non-cardiovascular  phase ii study combined methotrexate and tenip...
3  non-cardiovascular  flow cytometric dna analysis parathyroid tumor...
4      cardiovascular  paraneoplastic vasculitic neuropathy treatable...


In [7]:
testData = pd.DataFrame({'medical_abstract': test['medical_abstract'], 'condition_label': test['condition_label']})

In [8]:
testData.head()

,medical_abstract,condition_label
0,obstructive sleep apnea following topical orop...,non-cardiovascular
1,neutrophil function and pyogenic infections bo...,non-cardiovascular
2,phase ii study combined methotrexate and tenip...,non-cardiovascular
3,flow cytometric dna analysis parathyroid tumor...,non-cardiovascular
4,paraneoplastic vasculitic neuropathy treatable...,cardiovascular


In [9]:
def convertLabels(x):
  return 0 if x == 'non-cardiovascular' else 1

testData['condition_label'] = testData['condition_label'].apply(convertLabels)
testData.head()

,medical_abstract,condition_label
0,obstructive sleep apnea following topical orop...,0
1,neutrophil function and pyogenic infections bo...,0
2,phase ii study combined methotrexate and tenip...,0
3,flow cytometric dna analysis parathyroid tumor...,0
4,paraneoplastic vasculitic neuropathy treatable...,1


In [10]:
testData = testData.astype({'medical_abstract': 'string', 'condition_label': 'int'})
print(testData.dtypes)

medical_abstract    string
condition_label      int64
dtype: object


In [11]:
testData = testData.to_records(index=False)

In [12]:
num_classes = 2
num_epochs = 101
batch_size = 32
hidden_dim = 256
max_length = 256

In [13]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, hidden_dim),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(hidden_dim, num_classes)
)
model.load_state_dict(torch.load("./models/biobert_fine_tuned_epoch_16.pt")['model_state_dict'])
model.to(device)
print()

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [30]:
import textattack

In [31]:
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)
attack = textattack.attack_recipes.bae_garg_2019.BAEGarg2019.build(model_wrapper)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [19]:
for example in testData[1:20]:
  text, label = example
  print("Original Text:")
  print(text, label)
  result = attack.attack(text, int(label))
  print("BertAttack Result: ")
  print(result.__str__(color_method='ansi'))
  print("\n\n")

Original Text:
neutrophil function and pyogenic infections bone marrow transplant recipients consecutive entry trial incidence and time course decreased neutrophil function assessed 20 patients treated with allogeneic bone marrow transplantation bmt aim study assess prognostic value low neutrophil function late pyogenic infections chemotaxis superoxide production and phagocyticbactericidal activity studied before and 2 6 9 and 12 months after bmt skin window migration quantitatively assessed 2 months after bmt infectious complications recorded prospectively with preset criteria during 1 year six 20 leukemic patients had defective neutrophil function before bmt two months after bmt all 10 patients with greater than stage ii graftversushost disease gvhd and 6 10 patients with less than or equal stage ii gvhd had at least one decreased function at this time patients with subsequent pyogenic infections had lower chemotaxis p less than 05 phagocyticbactericidal activity p less than 005 and 

In [22]:
example = ('supravalvar aortic stenosis complication aortic valve surgery after patient had aortic valvotomy and aortotomy relief valvar aortic stenosis supravalvar stenosis developed over next six years', 1)
text, label = example
print("Original Text:")
print(text, label)
result = attack.attack(text, int(label))
print("BertAttack Result: ")
print(result.__str__(color_method='ansi'))
print("\n\n")

Original Text:
supravalvar aortic stenosis complication aortic valve surgery after patient had aortic valvotomy and aortotomy relief valvar aortic stenosis supravalvar stenosis developed over next six years 1
BertAttack Result: 
1 (64%) --> 0 (51%)

supravalvar aortic stenosis complication aortic valve surgery after patient had aortic valvotomy and aortotomy relief valvar aortic stenosis supravalvar stenosis developed over next six years

supravalvar node stenosis complication aortic valve surgery after patient had aortic valvotomy and aortotomy relief valvar ius stenosis supravalvar stenosis developed over next six years





In [23]:
example = ('dynamic cardiomyoplasty chronic chagas heart disease clinicopathological data we report 44yearold man with chronic chagasic cardiomyopathy who underwent latissimus dorsi dynamic cardiomyoplasty and died 4 months later clinicopathological findings discussed and literature reviewed', 1)
text, label = example
print("Original Text:")
print(text, label)
result = attack.attack(text, int(label))
print("BertAttack Result: ")
print(result.__str__(color_method='ansi'))
print("\n\n")

Original Text:
dynamic cardiomyoplasty chronic chagas heart disease clinicopathological data we report 44yearold man with chronic chagasic cardiomyopathy who underwent latissimus dorsi dynamic cardiomyoplasty and died 4 months later clinicopathological findings discussed and literature reviewed 1
BertAttack Result: 
1 (72%) --> 0 (70%)

dynamic cardiomyoplasty chronic chagas heart disease clinicopathological data we report 44yearold man with chronic chagasic cardiomyopathy who underwent latissimus dorsi dynamic cardiomyoplasty and died 4 months later clinicopathological findings discussed and literature reviewed

dynamic current chronic chagas heart disease clinicopathological data we report 44yearold man with chronic chagasic cardiomyopathy who underwent latissimus dorsi dynamic movement and died 4 months later clinicopathological findings discussed and literature reviewed





In [17]:
example = ('giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium', 1)
text, label = example
print("Original Text:")
print(text, label)
result = attack.attack(text, int(label))
print("BertAttack Result: ")
print(result.__str__(color_method='ansi'))
print("\n\n")

Original Text:
giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium 1
BertAttack Result: 
1 (84%) --> 0 (52%)

giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium

giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest neck cardiac enlargement due thick left atrium that distorted cardiac structures echocardiogram and firstpass nuclear cells able replicate huge left atrium





In [36]:
example = ('sudden death from acute cocaine intoxication virginia 1988 review medical examiners autopsy records revealed that 1988 33 residents virginia died acute cocaine intoxication subjects 19 45 years old most men used cocaine intravenously and used other toxic substances with lethal dose cocaine', 0)
text, label = example
print("Original Text:")
print(text, label)
result = attack.attack(text, int(label))
print("BertAttack Result: ")
print(result.__str__(color_method='ansi'))
print("\n\n")

Original Text:
sudden death from acute cocaine intoxication virginia 1988 review medical examiners autopsy records revealed that 1988 33 residents virginia died acute cocaine intoxication subjects 19 45 years old most men used cocaine intravenously and used other toxic substances with lethal dose cocaine 0
BertAttack Result: 
0 (71%) --> [FAILED]

sudden death from acute cocaine intoxication virginia 1988 review medical examiners autopsy records revealed that 1988 33 residents virginia died acute cocaine intoxication subjects 19 45 years old most men used cocaine intravenously and used other toxic substances with lethal dose cocaine





In [44]:
testData = pd.DataFrame({'medical_abstract': test['medical_abstract'], 'condition_label': test['condition_label']})
testData.head()

,medical_abstract,condition_label
0,obstructive sleep apnea following topical orop...,non-cardiovascular
1,neutrophil function and pyogenic infections bo...,non-cardiovascular
2,phase ii study combined methotrexate and tenip...,non-cardiovascular
3,flow cytometric dna analysis parathyroid tumor...,non-cardiovascular
4,paraneoplastic vasculitic neuropathy treatable...,cardiovascular


In [45]:
shortTestData = testData[testData['medical_abstract'].str.len() < 300]
shortTestData.head()

,medical_abstract,condition_label
61,ptca gastroepiploic bypass technique percutane...,non-cardiovascular
75,catheterization coronary artery bypass graft f...,non-cardiovascular
100,gastroesophageal reflux infants clinically sig...,non-cardiovascular
110,sudden death from acute cocaine intoxication v...,non-cardiovascular
119,case unilateral posterior ischemic optic neuro...,cardiovascular


In [46]:
shortTestData['condition_label'] = shortTestData['condition_label'].apply(convertLabels)
shortTestData.head()

/tmp/ipykernel_446/1757435916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortTestData['condition_label'] = shortTestData['condition_label'].apply(convertLabels)


,medical_abstract,condition_label
61,ptca gastroepiploic bypass technique percutane...,0
75,catheterization coronary artery bypass graft f...,0
100,gastroesophageal reflux infants clinically sig...,0
110,sudden death from acute cocaine intoxication v...,0
119,case unilateral posterior ischemic optic neuro...,1


In [47]:
len(shortTestData)

56

In [51]:
nonCardiovascularExamples = pd.DataFrame(shortTestData[shortTestData['condition_label'] == 0])
len(nonCardiovascularExamples)

43

In [52]:
nonCardiovascularExamples = nonCardiovascularExamples.to_records(index=False)

In [53]:
for example in nonCardiovascularExamples:
    text, label = example
    print("Original Text:")
    print(text, label)
    result = attack.attack(text, int(label))
    print("BertAttack Result: ")
    print(result.__str__(color_method='ansi'))
    print("\n\n")

Original Text:
ptca gastroepiploic bypass technique percutaneous transluminal coronary angioplasty ptca gastroepiploic bypass described using standard ptca devices severe spasm gastroepiploic bypass occurred modification guide catheter position suggested avoid inducing gastroepiploic bypass spasm 0
BertAttack Result: 
1 (68%) --> [38:5:240m[SKIPPED]

ptca gastroepiploic bypass technique percutaneous transluminal coronary angioplasty ptca gastroepiploic bypass described using standard ptca devices severe spasm gastroepiploic bypass occurred modification guide catheter position suggested avoid inducing gastroepiploic bypass spasm



Original Text:
catheterization coronary artery bypass graft from descending aorta increasing frequency reoperation coronary artery disease has led use variety grafts this report describes catheter technique selective opacification saphenous vein graft from descending thoracic aorta posterior coronary circulation 0
BertAttack Result: 
1 (69%) --> [38:5:240m[

BertAttack Result: 
0 (87%) --> [FAILED]

procainamideinduced respiratory insufficiency after cardiopulmonary bypass 46yearold man could not be weaned from ventilatory support while receiving procainamide when drug discontinued patient successfully weaned shortly thereafter



Original Text:
influence menstrual cycle systemic diseases physiological changes associated with menstrual cycle influence clinical course some diseases such as bronchial asthma allergies anaphylaxis epilepsy migraine dermatoses and porphyria hormonal manipulation can be beneficial some patients 0
BertAttack Result: 
0 (95%) --> [FAILED]

influence menstrual cycle systemic diseases physiological changes associated with menstrual cycle influence clinical course some diseases such as bronchial asthma allergies anaphylaxis epilepsy migraine dermatoses and porphyria hormonal manipulation can be beneficial some patients



Original Text:
pituitary hemorrhage into rathkes cleft cyst this report describes case symptomat

BertAttack Result: 
0 (98%) --> [FAILED]

cervical spine diastematomyelia adulthood adult cervical diastematomyelia rare malformation and usually associated with posterior element anomalies case isolated cervical diastematomyelia which initially thought be herniated cervical disc described diagnosis and surgical management discussed



Original Text:
accessory armdysraphism or disparity case report case 3monthold infant with accessory third arm reported extra appendage attached at midcervical region and associated with posterior cervical dysraphism and cervical cord lipoma possible theories origin examined 0
BertAttack Result: 
0 (99%) --> [FAILED]

accessory armdysraphism or disparity case report case 3monthold infant with accessory third arm reported extra appendage attached at midcervical region and associated with posterior cervical dysraphism and cervical cord lipoma possible theories origin examined



Original Text:
pituitary hemorrhage into rathkes cleft cyst this report descri

BertAttack Result: 
0 (98%) --> [FAILED]

7th nerve palsy after extradural blood patch we describe patient who developed 7th cranial nerve palsy following extradural blood patch full recovery followed likely aetiology discussed



Original Text:
lipomatous hypertrophy interatrial septum unusual intraoperative finding lipomatous hypertrophy interatrial septum incidental finding man 45 undergoing coronary artery bypass grafting unstable angina he not overweight and did not have any rhythm disturbances diagnosis made frozen section 0
BertAttack Result: 
1 (66%) --> [38:5:240m[SKIPPED]

lipomatous hypertrophy interatrial septum unusual intraoperative finding lipomatous hypertrophy interatrial septum incidental finding man 45 undergoing coronary artery bypass grafting unstable angina he not overweight and did not have any rhythm disturbances diagnosis made frozen section



Original Text:
multiple sclerosis and narcolepsy possible similar genetic susceptibility we have studied 2 patients w

In [19]:
text = ["giant left lung case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left lung that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left lung"]
labels = ['cardiovascular']
with torch.no_grad():
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=max_length, truncation=True) for v in text]).cuda()
    a_mask = (tv!=0).type(torch.int64).cuda()
    outputs = model(tv,attention_mask=a_mask)[0].cpu().numpy()
    print(np.argmax(outputs))

1
